In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer

2023-06-07 15:08:27.330240: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-07 15:08:27.611374: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-07 15:08:27.611400: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-07 15:08:27.647193: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-07 15:08:28.508387: W tensorflow/stream_executor/platform/de

In [12]:
data = pd.read_csv('../raw_data/reddit_all_symbols.csv',)
data.head(3)

,symbol,content,created_at,score
0,IPG,Your Morning Coffee: 10/05/2018 #EARNINGS\n\n ...,2018-10-05,243.0
1,IPG,Tendiegenerator.java import java.util.Arra...,2020-10-04,185.0
2,IPG,J.P. Morgan Early Look at the Market – Mon 10....,2017-10-16,145.0


In [53]:
import re
cleanr = re.compile('<.*?>')
data['content'] = data['content'].apply(lambda x: re.sub('[0-9]+', '', re.sub(r'http\S+', '', re.sub(cleanr, '', x))))
#rem_url=re.sub(r'http\S+', '',cleantext)
#rem_num = re.sub('[0-9]+', '', rem_url)
data['content'] = data['content'].apply(lambda x : x.lower().replace('\n', ' ').strip())
#data['content'] = data['content'].apply(lambda x: re.sub('[0-9]+', '', x))
X = data.loc[:20, ['symbol', 'content']]
X['content'] = X['content'].apply(lambda x: x[0:512])

In [71]:
X = data.loc[:, ['symbol', 'content']]
X['content'] = X['content'].apply(lambda x: x[0:512])
X

,symbol,content
0,IPG,your morning coffee: // #earnings costco (c...
1,IPG,tendiegenerator.java import java.util.arra...
2,IPG,j.p. morgan early look at the market – mon .. ...
3,IPG,wall street week ahead for the trading week be...
4,IPG,your morning coffee [wednesday ..] #earnings ...
...,...,...
212310,FIVE,explain to me like i'm five. is hmny really to...
212311,FIVE,"after holding for five years, selling $fb toda..."
212312,FIVE,oracle just somehow jumped % in five minutes? ...
212313,FIVE,five of nine seems fine for usa mines &#xb; [...


In [44]:
tokenizer = Tokenizer()

# The tokenization learns a dictionary that maps a token (integer) to each word
# It can be done only on the train set - we are not supposed to know the test set!
# This tokenization also lowercases your words, apply some filters, and so on - you can check the doc if you want
text = tokenizer.fit_on_texts(X['content'])
    
# We apply the tokenization to the train and test set
X_token = tokenizer.texts_to_sequences(X['content'])
#X_test_token = tokenizer.texts_to_sequences(X_test)
tokenizer.num_words

In [46]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 14.9 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 16.4 MB/s eta 0:00:00m eta 0:00:010:01:01


In [72]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

model = BertForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis",num_labels=3)
tokenizer = BertTokenizer.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")

nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

sentences = ["Operating profit rose to EUR 13.1 mn from EUR 8.7 mn in the corresponding period in 2007 representing 7.7 % of net sales.",  
             "Bids or offers include at least 1,000 shares and the value of the shares must correspond to at least EUR 4,000.", 
             "Raute reported a loss per share of EUR 0.86 for the first half of 2009 , against EPS of EUR 0.74 in the corresponding period of 2008.", 
             ]
sentences = list(X['content'])
#print(sentences)
results = nlp(sentences)
results



KeyboardInterrupt: 

In [16]:
# Appl
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

X = pd.read_csv('../raw_data/industries/AAPL.csv')
X = X.loc[:, ['symbol', 'content', 'created_at', 'score']]
X['content'] = X['content'].apply(lambda x: x[0:512])
X

model = BertForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis",num_labels=3)
tokenizer = BertTokenizer.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")

nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

sentences = ["Operating profit rose to EUR 13.1 mn from EUR 8.7 mn in the corresponding period in 2007 representing 7.7 % of net sales.",  
             "Bids or offers include at least 1,000 shares and the value of the shares must correspond to at least EUR 4,000.", 
             "Raute reported a loss per share of EUR 0.86 for the first half of 2009 , against EPS of EUR 0.74 in the corresponding period of 2008.", 
             ]
sentences = list(X['content'])
#print(sentences)
results = nlp(sentences)
results

[{'label': 'neutral', 'score': 0.9915090799331665},
 {'label': 'neutral', 'score': 0.9991363883018494},
 {'label': 'neutral', 'score': 0.9667267799377441},
 {'label': 'positive', 'score': 0.9967592358589172},
 {'label': 'negative', 'score': 0.9984101057052612},
 {'label': 'neutral', 'score': 0.9992467164993286},
 {'label': 'neutral', 'score': 0.997963547706604},
 {'label': 'neutral', 'score': 0.9980545043945312},
 {'label': 'neutral', 'score': 0.9985528588294983},
 {'label': 'neutral', 'score': 0.9962025284767151},
 {'label': 'neutral', 'score': 0.9997654557228088},
 {'label': 'positive', 'score': 0.9996376037597656},
 {'label': 'neutral', 'score': 0.9941148161888123},
 {'label': 'neutral', 'score': 0.9996575117111206},
 {'label': 'neutral', 'score': 0.999605119228363},
 {'label': 'neutral', 'score': 0.9995415210723877},
 {'label': 'neutral', 'score': 0.9959171414375305},
 {'label': 'neutral', 'score': 0.9944250583648682},
 {'label': 'neutral', 'score': 0.9967824220657349},
 {'label': 

In [17]:
sentiment = {
    'negative': -1,
    'neutral': 0,
    'positive': 1,
}
d = pd.DataFrame(results)
X.loc[:, 'sentiment'] = d['label'].apply(lambda x: sentiment[x])
X[['symbol', 'score', 'created_at', 'sentiment']]
X

,symbol,content,created_at,score,sentiment
0,AAPL,Bears feasting on AAPL next week confirmed,2020-09-26,21052,0
1,AAPL,Someone purchased $1.3 BILLION worth of $AAPL ...,2022-12-05,20775,0
2,AAPL,Apple started WWIII so that phones would be de...,2020-01-04,15905,0
3,AAPL,AAPL's innovative product strategy,2019-01-03,11882,1
4,AAPL,Chinese Foxconn workers facing off against pol...,2022-11-24,10471,-1
...,...,...,...,...,...
227,AAPL,$AAPL put the rest of my tiny portfolio on 152...,2022-09-30,19,0
228,AAPL,"How do you feel about aapl next week? Man, thi...",2023-04-29,17,1
229,AAPL,Talking about AAPL positions! Holding 11/20 12...,2020-09-08,13,0
230,AAPL,How likely will AAPL hit 470 by Friday? Bought...,2020-08-19,11,0


In [18]:
X.to_csv('../raw_data/reddit_sentimental_AAPL.csv')